In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Model Evaluation and optimization

In the steps of machine learning you:
1. take data
2. build a model
3. evaluate a model
4. optimize it
5. rebuild if needed
Then lookst at new data and the prediction

### Accuracy

Usually the main goal is to make accurate predictions. One measure of performance on a model is how well tha t5model will perform on new data. 

#### Overfitting & model optimism

#### Cross-validation

holdout method because a random subset of the training data is held out from the training process. So you use a training subset to fit the model and only the testing subset to evaluate the accuracy of the model. 

Leave out 20 - 40$ of the data as the testing subset. 



In [2]:
#make up some data
features = rand(100,5)
target = rand(100) > 0.5

In [ ]:
#holdout method
N = features.shape[0]
N_train = floor(0.7 * N)

